In [1]:
%%capture
!pip install -U langchain langchain_experimental openai

In [ ]:
!pip install -U langchain-openai

In [3]:
from langchain_openai import ChatOpenAI

In [4]:
# set environment variables
# https://platform.openai.com/account/api-keys
import os
os.environ["OPENAI_API_KEY"] = "<your key>"

In [5]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.pydantic_v1 import BaseModel
from langchain_experimental.tabular_synthetic_data.base import SyntheticDataGenerator
from langchain_experimental.tabular_synthetic_data.openai import create_openai_data_generator, OPENAI_TEMPLATE
from langchain_experimental.tabular_synthetic_data.prompts import SYNTHETIC_FEW_SHOT_SUFFIX, SYNTHETIC_FEW_SHOT_PREFIX

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:

from typing import Optional, Literal

from pydantic import Field, BaseModel

class telecomcustomer(BaseModel):
    short_desc: str = Field(description="Short description of the incident which includes short summary")
    description: str = Field(description="Detailed description of the incident in the network")
    activities: str = Field(description="Activities taken to resolve the incident by multiple teams.")
    resolution_notes: str = Field(description="Notes on the resolution of the issue to understand how it was resolved.")





In [7]:
examples = [
    {"example": """short description: Network Interruption on OLT Card Port SINGSGKGEX0/HUTQ, Description:A network interruption was detected on port SINGSGKGEX0/HUTQ of the Optical Line Terminal (OLT) card at 14:30 local time. Customers in the affected area reported internet connectivity issues, indicating a potential failure at the network hardware level, Activities:Alert received from network monitoring system indicating a port failure.Confirmed customer complaints in the affected area through service desk reports.Technician dispatched to the site to perform physical inspection of the OLT card and connected fibers.On-site technician reported no visible physical damage but suspected a faulty port or loose connection.Performed remote diagnostic tests on the OLT card confirming suspicions of a port defect.Decision made to reroute the network traffic to an available backup port to minimize downtime.,Resolution notes:Re-routed affected traffic through backup portSINGSGKGEX0/HUTP successfully.Customers began reporting restoration of internet services. Monitoring systems showed stable metrics.Replaced the defective port hardware at SINGSGKGEX0/HUTQ. Tested new setup successfully.Full network functionality restored. Monitored the system for an additional hour to ensure stability. Incident closed. """},
    {"example": """short description: Signal Degradation on OLT Card Port SINGSGKGEX1/JVZY, Description:On October 14, 2023, at approximately 09:25 local time, a significant signal degradation was detected on port SINGSGKGEX1/JVZY of the Optical Line Terminal (OLT) card. This affected internet performance for customers in the northern district, causing intermittent connectivity and reduced bandwidth., Activities:Received multiple alerts from the network monitoring system indicating signal degradation on the port. , Resolution notes: The incident was resolved by identifying and removing contamination on the fiber optic connectors serving port 2/3/5 of SINGSGKGEX0/HUTQ C300/4 OLT. The contamination was causing signal degradation that worsened during high traffic periods. After cleaning the optical interfaces and resetting the port, optical power levels improved from -19.5dBm to -14.2dBm, restoring full bandwidth capacity to all 96 business customers. As a preventive measure, the site has been added to the priority cleaning schedule with quarterly fiber connection inspection and cleaning. Additionally, monitoring thresholds have been adjusted to trigger alerts when optical power levels approach -18dBm rather than waiting until the -20dBm critical threshold"""},

]

In [8]:
OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix="Please generate the short description ,description and activities for a network incident in telecommunication industry that is on the OLT card or one off the OLT port and can impact multiple customers.Please do return real data.",
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)

In [9]:
synthetic_data_generator = create_openai_data_generator(
    output_schema=telecomcustomer,
    llm=ChatOpenAI(temperature=1),
    prompt=prompt_template,
)

<ipython-input-9-f7032420c87e>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(temperature=1),


In [10]:
synthetic_results = synthetic_data_generator.generate(
    subject="telecom customer",
    extra="Generate unique data everytime.",
    runs=1000,
)

In [11]:
type(synthetic_results)

list

In [12]:
synthetic_results

[telecomcustomer(short_desc='Network Interruption on OLT Card Port SINGSGKGEX2/KVLM', description='A network interruption was detected on port SINGSGKGEX2/KVLM of the Optical Line Terminal (OLT) card at 11:45 local time. Customers in the impacted region experienced sudden internet disconnections and slow connectivity, indicating a potential issue with the specific OLT port.', activities='Alert triggered by network monitoring system upon port failure detection. Customer complaints verified via service desk reports. Field technician dispatched for physical inspection of the OLT card and fiber connections. On-site evaluation revealed no visible damage but suspected port malfunction or loose connection. Remote diagnostic tests conducted, confirming port malfunction.Visual inspection showed no significant issues. Decision made to reroute traffic to backup port for minimal service disruption.', resolution_notes='Successfully re-routed network traffic to backup port SINGSGKGEX2/KVLP. Customer

In [13]:
import pandas as pd

# Create a list of dictionaries from the objects
synthetic_data = []
for item in synthetic_results:
    synthetic_data.append({
        'short_desc': item.short_desc,
        'description': item.description,
        'activities': item.activities,
        'resolution_notes': item.resolution_notes

    })

# Create a Pandas DataFrame from the list of dictionaries
synthetic_df = pd.DataFrame(synthetic_data)

# Display the DataFrame
print(type(synthetic_df))
synthetic_df

<class 'pandas.core.frame.DataFrame'>


,short_desc,description,activities,resolution_notes
0,Network Interruption on OLT Card Port SINGSGKG...,A network interruption was detected on port SI...,Alert triggered by network monitoring system u...,Successfully re-routed network traffic to back...
1,Signal Degradation on OLT Card Port SINGSGKGEX...,"On March 28, 2024, at 14:30 local time, a sign...",Network monitoring system generated alerts upo...,The issue was resolved by addressing the fiber...
2,Network Interruption on OLT Card Port SINGSGKG...,A network interruption was detected on port SI...,Alert triggered by network monitoring system u...,Successfully re-routed network traffic to back...
3,Signal Degradation on OLT Card Port SINGSGKGEX...,"On May 12, 2024, at 09:15 local time, a signal...",Network monitoring system generated alerts upo...,The issue was resolved by replacing the damage...
4,Network Interruption on OLT Card Port SINGSGKG...,An unexpected network interruption occurred on...,Automatic alert triggered by the network monit...,Successfully rerouted network traffic to backu...
...,...,...,...,...
995,Network Incident on OLT Card Port XYZ123,"At 1:30 PM on September 5, 2047, a network inc...",Upon identification of the network incident on...,The network incident on OLT Card Port XYZ123 h...
996,Network Incident on OLT Card Port DEF456,"At 3:15 PM on January 20, 2048, a network inci...",Upon detection of the network incident on port...,The network incident on OLT Card Port DEF456 h...
997,Network Incident on OLT Card Port ABC789,"At 10:45 AM on May 12, 2049, a network inciden...",Upon identification of the network incident on...,The network incident on OLT Card Port ABC789 h...
998,Network Incident on OLT Card Port LMN123,"At 2:30 PM on August 5, 2050, a network incide...",Upon detection of the network incident on port...,The network incident on OLT Card Port LMN123 h...


In [14]:
# prompt: how to generate csv file from dataframe

# Export the DataFrame to a CSV file
synthetic_df.to_csv('synthetic_data.csv', index=False)


### Extractors